## Sensitivity Analysis for Linear Regression Models
This method is based on https://carloscinelli.com/files/Cinelli%20and%20Hazlett%20(2020)%20-%20Making%20Sense%20of%20Sensitivity.pdf 

### Step 1: Load required packages

In [ ]:
import os, sys
sys.path.append(os.path.abspath("../../../"))
import dowhy
from dowhy import CausalModel
import pandas as pd
import numpy as np
import dowhy.datasets 

# Config dict to set the logging level
import logging.config
DEFAULT_LOGGING = {
    'version': 1,
    'disable_existing_loggers': False,
    'loggers': {
        '': {
            'level': 'WARN',
        },
    }
}

logging.config.dictConfig(DEFAULT_LOGGING)
# Disabling warnings output
import warnings
from sklearn.exceptions import DataConversionWarning
warnings.filterwarnings(action='ignore', category=DataConversionWarning)

### Step 2: Load the dataset 
We create a dataset with linear relationships between common causes and treatment, and common causes and outcome,

In [ ]:
np.random.seed(100) 
data = dowhy.datasets.linear_dataset( beta = 10,
                                      num_common_causes = 7,
                                      num_samples = 100,
                                      num_treatments = 1,
                                     stddev_treatment_noise=10
                                    )

In [ ]:
model = CausalModel(
            data=data["df"],
            treatment=data["treatment_name"],
            outcome=data["outcome_name"],
            graph=data["gml_graph"],
            test_significance=None,
        )
model.view_model()
from IPython.display import Image, display
display(Image(filename="causal_model.png"))
data['df'].head()

### Step 3: Create Causal Model
Remove one of the common causes to simulate unobserved confounding

In [ ]:
data["df"] = data["df"].drop("W4", axis = 1)
graph_str = 'graph[directed 1node[ id "y" label "y"]node[ id "W0" label "W0"] node[ id "W1" label "W1"] node[ id "W2" label "W2"] node[ id "W3" label "W3"] node[ id "W5" label "W5"] node[ id "W6" label "W6"]node[ id "v0" label "v0"]edge[source "v0" target "y"]edge[ source "W0" target "v0"] edge[ source "W1" target "v0"] edge[ source "W2" target "v0"] edge[ source "W3" target "v0"] edge[ source "W5" target "v0"] edge[ source "W6" target "v0"]edge[ source "W0" target "y"] edge[ source "W1" target "y"] edge[ source "W2" target "y"] edge[ source "W3" target "y"] edge[ source "W5" target "y"] edge[ source "W6" target "y"]]'
model = CausalModel(
            data=data["df"],
            treatment=data["treatment_name"],
            outcome=data["outcome_name"],
            graph=graph_str,
            test_significance=None,
        )
model.view_model()
from IPython.display import Image, display
display(Image(filename="causal_model.png"))
data['df'].head()

### Step 4: Identification

In [ ]:
identified_estimand = model.identify_effect(proceed_when_unidentifiable=True)
print(identified_estimand)

### Step 5: Estimation
Currently only Linear Regression estimator is supported for Linear Sensitivity Analysis

In [ ]:
estimate = model.estimate_effect(identified_estimand,method_name="backdoor.linear_regression")

### Step 6: Refutation and Sensitivity Analysis

In [ ]:
refute = model.refute_estimate(identified_estimand, estimate , method_name = "add_unobserved_common_cause",
                              simulated_method_name = "PartialR2", benchmark_covariates = ["W2"],
                              kd = [0.04, 0.08])

The partial R^2 of treatment with outcome shows how strongly confounders explaining all the residual outcome variation would have to be associated with the treatment to eliminate the estimated effect.<br>
The robustness value measures the minimum strength of association unobserved confounding should have with bot treatment and outcome, to change the conclusions.

In [ ]:
refute.stats

In [ ]:
refute.bounds_result

In [ ]:
refute.plot(sensitivity_variable = 't-value')